In [ ]:
import torch

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig, TrainingArguments
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training

In [ ]:
dataset = load_dataset("imdb", split="train")

In [ ]:
model_name = "tiiuae/falcon-7b-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model.config.use_cache = False

In [ ]:
lora_alpha = 32
lora_dropout = 0.05
lora_rank = 32

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(prepare_model_for_kbit_training(model), peft_config)

In [ ]:
output_dir = "xxx"
per_device_train_batch_size = 16
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 180 #100 #500
warmup_ratio = 0.03
lr_scheduler_type = "cosine"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
trainer = SFTTrainer(
    peft_model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
trainer.train()

In [ ]:
input_ids = tokenizer("My favorite movie is ", return_tensors="pt").input_ids.to("cuda")

In [ ]:
outputs = peft_model.generate(
    input_ids=input_ids,
    do_sample=False,
    max_length=32,
    use_cache=False,
    temperature=0.1,
    top_p=0.1,
    repetition_penalty=1.2,
    num_return_sequences=1,
)

In [ ]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
outputs = model.generate(
    input_ids=tokenizer("Why hello there, my little friend.  I ", return_tensors="pt").input_ids.to("cuda"),
    do_sample=False,
    max_length=32,
    use_cache=False,
    temperature=0.1,
    top_p=0.1,
    repetition_penalty=1.2,
    num_return_sequences=1,
)

In [ ]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)